In [63]:
import torch
from email.header import Header
from operator import index
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
import torch
import random
import scipy.spatial
import scipy.io


def create_dataset(
    dataset_folder,
    dataset_name,
    val_size,
    gt,
    horizon,
    delim="\t",
    train=True,
    eval=False,
    verbose=False,
):

    if train == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "train"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "train")
    if train == False and eval == False:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "val"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "val")
    if train == False and eval == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "test"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "test")

    datasets_list = datasets_list

    data = {}
    data_src = []
    data_trg = []
    data_seq_start = []
    data_frames = []
    data_dt = []
    data_peds = []

    val_src = []
    val_trg = []
    val_seq_start = []
    val_frames = []
    val_dt = []
    val_peds = []

    if verbose:
        print("start loading dataset")
        print("validation set size -> %i" % (val_size))

    for i_dt, dt in enumerate(datasets_list):
        if verbose:
            print("%03i / %03i - loading %s" % (i_dt + 1, len(datasets_list), dt))
        # if i_dt == 0 or i_dt == 1:
        # data = pd.read_csv(os.path.join(full_dt_folder, dt), sep="\t", header=None)
        # print(data.info())
        # print(data.head())
        raw_data = pd.read_csv(
            os.path.join(full_dt_folder, dt),
            delimiter=delim,
            names=[
                "frame",
                "ped",
                "x",
                "y",
                "scale",
                "head_x",
                "head_y",
                "l_ear_x",
                "l_ear_y",
                "l_elbow_x",
                "l_elbow_y",
                "l_eye_x",
                "l_eye_y",
                "l_foot_x",
                "l_foot_y",
                "l_hand_x",
                "l_hand_y",
                "l_hip_x",
                "l_hip_y",
                "l_knee_x",
                "l_knee_y",
                "l_shoulder_x",
                "l_shoulder_y",
                "neck_x",
                "neck_y",
                "r_ear_x",
                "r_ear_y",
                "r_elbow_x",
                "r_elbow_y",
                "r_eye_x",
                "r_eye_y",
                "r_foot_x",
                "r_foot_y",
                "r_hand_x",
                "r_hand_y",
                "r_hip_x",
                "r_hip_y",
                "r_knee_x",
                "r_knee_y",
                "r_shoulder_x",
                "r_shoulder_y",
                "R_1",
                "R_2",
                "R_3",
                "T_1",
                "T_2",
                "T_3",
            ],
            usecols=list(range(47)),
            na_values="?",
        )

        # print(raw_data.columns)
        raw_data.sort_values(by=["frame", "ped"], inplace=True)

        inp, out, info = get_strided_data_clust(raw_data, gt, horizon, 1)

        dt_frames = info["frames"]
        dt_seq_start = info["seq_start"]
        dt_dataset = np.array([i_dt]).repeat(inp.shape[0])
        dt_peds = info["peds"]

        if val_size > 0 and inp.shape[0] > val_size * 2.5:
            if verbose:
                print("created validation from %s" % (dt))
            k = random.sample(np.arange(inp.shape[0]).tolist(), val_size)
            val_src.append(inp[k, :, :])
            val_trg.append(out[k, :, :])
            val_seq_start.append(dt_seq_start[k, :, :])
            val_frames.append(dt_frames[k, :])
            val_dt.append(dt_dataset[k])
            val_peds.append(dt_peds[k])
            inp = np.delete(inp, k, 0)
            out = np.delete(out, k, 0)
            dt_frames = np.delete(dt_frames, k, 0)
            dt_seq_start = np.delete(dt_seq_start, k, 0)
            dt_dataset = np.delete(dt_dataset, k, 0)
            dt_peds = np.delete(dt_peds, k, 0)
        elif val_size > 0:
            if verbose:
                print(
                    "could not create validation from %s, size -> %i"
                    % (dt, inp.shape[0])
                )

        data_src.append(inp)
        data_trg.append(out)
        data_seq_start.append(dt_seq_start)
        data_frames.append(dt_frames)
        data_dt.append(dt_dataset)
        data_peds.append(dt_peds)

    data["src"] = np.concatenate(data_src, 0)
    data["trg"] = np.concatenate(data_trg, 0)
    data["seq_start"] = np.concatenate(data_seq_start, 0)
    data["frames"] = np.concatenate(data_frames, 0)
    data["dataset"] = np.concatenate(data_dt, 0)
    data["peds"] = np.concatenate(data_peds, 0)
    data["dataset_name"] = datasets_list

    mean = data["src"].mean((0, 1))
    std = data["src"].std((0, 1))

    if val_size > 0:
        data_val = {}
        data_val["src"] = np.concatenate(val_src, 0)
        data_val["trg"] = np.concatenate(val_trg, 0)
        data_val["seq_start"] = np.concatenate(val_seq_start, 0)
        data_val["frames"] = np.concatenate(val_frames, 0)
        data_val["dataset"] = np.concatenate(val_dt, 0)
        data_val["peds"] = np.concatenate(val_peds, 0)

        return IndividualTfDataset(data, "train", mean, std), IndividualTfDataset(
            data_val, "validation", mean, std
        )
    # print("----------")
    # print("src")
    # print(data["src"][0])
    # print("-------")
    # print("trg")
    # print(data["trg"][0])
    return IndividualTfDataset(data, "train", mean, std), None

    return IndividualTfDataset(data, "train", mean, std), IndividualTfDataset(
        data_val, "validation", mean, std
    )


class IndividualTfDataset(Dataset):
    def __init__(self, data, name, mean, std):
        super(IndividualTfDataset, self).__init__()

        self.data = data
        self.name = name

        self.mean = mean
        self.std = std

    def __len__(self):
        return self.data["src"].shape[0]

    def __getitem__(self, index):
        return {
            "src": torch.Tensor(self.data["src"][index]),
            "trg": torch.Tensor(self.data["trg"][index]),
            "frames": self.data["frames"][index],
            "seq_start": self.data["seq_start"][index],
            "dataset": self.data["dataset"][index],
            "peds": self.data["peds"][index],
        }


def create_folders(baseFolder, datasetName):
    try:
        os.mkdir(baseFolder)
    except:
        pass

    try:
        os.mkdir(os.path.join(baseFolder, datasetName))
    except:
        pass



def get_strided_data_clust(dt, gt_size, horizon, step):
    inp_te = []
    dtt = dt.astype(np.float32)
    raw_data = dtt
    # print("-----------")
    # print("In strided data --> raw data")
    # print(raw_data.shape)
    # print("raw data 0")
    # print(raw_data.iloc[0])
    ped = raw_data.ped.unique()
    # print(ped)
    frame = []
    ped_ids = []
    # print(dt)
    # print(gt_size)
    # print(horizon)
    # print(step)
    for p in ped:
        # print(f"p is  {p}")
        # print("u p in ped")
        # print(raw_data[raw_data.ped == p].shape)

        # print(1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step)

        for i in range(
            1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step
        ):

            frame.append(
                dt[dt.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, [0]]
                .values.squeeze()
            )
            # print("%i,%i,%i" % (i * 4, i * 4 + gt_size, i * 4 + gt_size + horizon))
            inp_te.append(
                raw_data[raw_data.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, 2:50]
                .values
            )
            ped_ids.append(p)

    frames = np.stack(frame)
    inp_te_np = np.stack(inp_te)
    ped_ids = np.stack(ped_ids)

    # inp_relative_pos= inp_te_np-inp_te_np[:,:1,:]
    inp_speed = np.concatenate(
        (
            np.zeros((inp_te_np.shape[0], 1, 2)),
            inp_te_np[:, 1:, 0:2] - inp_te_np[:, :-1, 0:2],
        ),
        1,
    )
    # inp_accel = np.concatenate((np.zeros((inp_te_np.shape[0],1,2)),inp_speed[:,1:,0:2] - inp_speed[:, :-1, 0:2]),1)
    # inp_std = inp_no_start.std(axis=(0, 1))
    # inp_mean = inp_no_start.mean(axis=(0, 1))
    # inp_norm= inp_no_start
    # inp_norm = (inp_no_start - inp_mean) / inp_std

    # vis=inp_te_np[:,1:,2:4]/np.linalg.norm(inp_te_np[:,1:,2:4],2,axis=2)[:,:,np.newaxis]
    # inp_norm=np.concatenate((inp_norm,vis),2)
    inp_norm = np.concatenate((inp_te_np, inp_speed), 2)
    inp_mean = np.zeros(4)
    inp_std = np.ones(4)

    return (
        inp_norm[:, :gt_size],
        inp_norm[:, gt_size:],
        {
            "mean": inp_mean,
            "std": inp_std,
            "seq_start": inp_te_np[:, 0:1, :].copy(),
            "frames": frames,
            "peds": ped_ids,
        },
    )


def distance_metrics(gt, preds):
    errors = np.zeros(gt.shape[:-1])

    for i in range(errors.shape[0]):
        for j in range(errors.shape[1]):
            errors[i, j] = scipy.spatial.distance.euclidean(gt[i, j], preds[i, j])
    return errors.mean(), errors[:, -1].mean(), errors
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformer.decoder import Decoder
from transformer.multihead_attention import MultiHeadAttention
from transformer.positional_encoding import PositionalEncoding
from transformer.pointerwise_feedforward import PointerwiseFeedforward
from transformer.encoder_decoder import EncoderDecoder
from transformer.encoder import Encoder
from transformer.encoder_layer import EncoderLayer
from transformer.decoder_layer import DecoderLayer
from transformer.batch import subsequent_mask
import numpy as np
import scipy.io
import os

import copy
import math

class IndividualTF(nn.Module):
    def __init__(self, enc_inp_size, dec_inp_size, dec_out_size, N=6,
                   d_model=512, d_ff=2048, h=8, dropout=0.1,mean=[0,0],std=[0,0]):
        super(IndividualTF, self).__init__()
        "Helper: Construct a model from hyperparameters."
        c = copy.deepcopy
        attn = MultiHeadAttention(h, d_model)
        ff = PointerwiseFeedforward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        self.mean=np.array(mean)
        self.std=np.array(std)
        self.model = EncoderDecoder(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            Decoder(DecoderLayer(d_model, c(attn), c(attn),
                                 c(ff), dropout), N),
            nn.Sequential(LinearEmbedding(enc_inp_size,d_model), c(position)),
            nn.Sequential(LinearEmbedding(dec_inp_size,d_model), c(position)),
            Generator(d_model, dec_out_size))

        # This was important from their code.
        # Initialize parameters with Glorot / fan_avg.
        for p in self.model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, *input):
        return self.model.generator(self.model(*input))

class LinearEmbedding(nn.Module):
    def __init__(self, inp_size,d_model):
        super(LinearEmbedding, self).__init__()
        # lut => lookup table
        self.lut = nn.Linear(inp_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, out_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, out_size)

    def forward(self, x):
        return self.proj(x)


device = torch.device("cuda")
model = IndividualTF(
        enc_inp_size=45,
        dec_inp_size=46,
        dec_out_size=46,
        N=6,
        d_model=512,
        d_ff=2048,
        h=8,
        dropout=0.1,
        mean=[0, 0],
        std=[0, 0],
    ).to(device)
PATH = r"C:\Users\alika\Desktop\research\transformer-location-prediction\models\Individual\fpl_train\00408.pth"
model.load_state_dict(torch.load(PATH))
model.eval()

IndividualTF(
  (model): EncoderDecoder(
    (encoder): Encoder(
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): MultiHeadAttention(
            (linears): ModuleList(
              (0): Linear(in_features=512, out_features=512, bias=True)
              (1): Linear(in_features=512, out_features=512, bias=True)
              (2): Linear(in_features=512, out_features=512, bias=True)
              (3): Linear(in_features=512, out_features=512, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PointerwiseFeedforward(
            (w_1): Linear(in_features=512, out_features=2048, bias=True)
            (w_2): Linear(in_features=2048, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (sublayer): ModuleList(
            (0): SublayerConnection(
              (norm): LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [66]:
test_dataset, _ = create_dataset(
        "datasets",
        "fpl",
        0,
        8,
        12,
        delim="\t",
        train=False,
        eval=True,
        verbose=True,
    )
test_dl = torch.utils.data.DataLoader(
        test_dataset, batch_size=100, shuffle=False, num_workers=0
    )
mean = torch.cat(
        (test_dataset[:]["src"][:, 1:, 2:47], test_dataset[:]["trg"][:, :, 2:47]), 1
    ).mean((0, 1))
    # std=train_dataset[:]['src'][:,1:,2:4].std((0,1))
std = torch.cat(
    (test_dataset[:]["src"][:, 1:, 2:47], test_dataset[:]["trg"][:, :, 2:47]), 1
).std((0, 1))
means = []
stds = []
for i in np.unique(test_dataset[:]["dataset"]):
    ind = test_dataset[:]["dataset"] == i
    means.append(
        torch.cat(
            (
                test_dataset[:]["src"][ind, 1:, 2:47],
                test_dataset[:]["trg"][ind, :, 2:47],
            ),
            1,
        ).mean((0, 1))
    )
    stds.append(
        torch.cat(
            (
                test_dataset[:]["src"][ind, 1:, 2:47],
                test_dataset[:]["trg"][ind, :, 2:47],
            ),
            1,
        ).std((0, 1))
    )

mean = torch.stack(means).mean(0)
std = torch.stack(stds).mean(0)

mean_list = mean.detach().tolist()
for i, tens in enumerate(mean_list):
    if tens == 0:
        mean_list[i] = 0.00000001

std_list = std.detach().tolist()
for i, tens in enumerate(std_list):
    if tens == 0:
        std_list[i] = 0.00000001

mean = torch.FloatTensor(mean_list)
std = torch.FloatTensor(std_list)

start loading dataset
validation set size -> 0
001 / 002 - loading GOPR0321U20_imputed.txt
002 / 002 - loading GOPR0324U20_imputed.txt


In [100]:
gt = []
pr = []
inp_ = []
peds = []
frames = []
dt = []

for id_b, batch in enumerate(test_dl):
    
    inp_.append(batch["src"])
    gt.append(batch["trg"][:, :, 0:2])
    frames.append(batch["frames"])
    peds.append(batch["peds"])
    dt.append(batch["dataset"])

    inp = (
        batch["src"][:, :, 2:47].to(device) - mean.to(device)
    ) / std.to(device)
    trg = (
        batch["trg"][:, :, 2:47].to(device) - mean.to(device)
    ) / std.to(device)
    print("inp i trg")
    print(inp[0, :, 2])
    print(trg[0, :, 2])
    src_att = torch.ones((inp.shape[0], 1, inp.shape[1])).to(device)

    start_of_seq = (
        torch.Tensor(
            [
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                1,
            ]
        )
        .unsqueeze(0)
        .unsqueeze(1)
        .repeat(inp.shape[0], 1, 1)
        .to(device)
    )
    dec_inp = start_of_seq
    
    for i in range(12):
        trg_att = (
            subsequent_mask(dec_inp.shape[1])
            .repeat(dec_inp.shape[0], 1, 1)
            .to(device)
        )
        out = model(inp, dec_inp, src_att, trg_att)
        print("-------")
        #print("inp complete")
        #print(inp[:, i, :][0])
        print("dec inp")
        print(dec_inp[:, i, :][0])
        print("trg complete")
        print(trg[:, i, :][0])
        print("out")
        print(out[:, -1, :][0])
        out_2 = out
        print("out 2 before")
        print(out_2[:, -1, :][0])
        
        out_2[:, -1, 2:45] = trg[:, i, 2:]
        #print(out_2[:, -1, 2:])
        print("out2 after")
        print(out_2[:, -1, :][0])
        
        
        
        
        
        dec_inp = torch.cat((dec_inp, out_2), 1)
        

    preds_tr_b = (
        dec_inp[:, 1:, 0:2] * std[:2].to(device) + mean[:2].to(device)
    ).detach().cpu().numpy().cumsum(1) + batch["src"][:, -1:, 0:2].detach().cpu().numpy()
    pr.append(preds_tr_b)
    

peds = np.concatenate(peds, 0)
frames = np.concatenate(frames, 0)
dt = np.concatenate(dt, 0)
gt = np.concatenate(gt, 0)
dt_names = test_dataset.data["dataset_name"]
pr = np.concatenate(pr, 0)
# print(type(gt))
# print("gt type gore, dole pr")
# print(type(pr))
#print(pr.shape)
#print(pr)
#print(gt)
mad, fad, errs = distance_metrics(gt, pr)

inp i trg
tensor([ 0.8144,  0.9976,  1.0028,  0.5401,  0.1723, -0.4663, -0.9288, -1.2019],
       device='cuda:0')
tensor([-1.5701, -1.3888, -1.2956, -1.3864, -1.5711, -1.6619, -1.3890, -1.2968,
        -1.2080, -1.5709, -1.5696, -1.3877], device='cuda:0')
-------
dec inp
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], device='cuda:0')
trg complete
tensor([-1.5582, -6.7953, -1.5701, -6.3599, -3.2488,  5.0117,  4.6867, -6.7495,
        -2.2241, -1.2661, -1.2760, -6.9972, -2.0054, -6.5960, -2.1782, -6.9817,
        -1.7961, -6.9122, -1.8294,  0.0491,  1.0467, -7.0822, -1.4201, -6.3629,
        -1.8793,  0.0566,  0.9013, -6.2828, -2.4123, -6.4094, -1.9951, -6.8383,
        -1.9422, -6.3595, -2.2232, -6.7463, -1.7100,  2.7731, -1.0680, -1.5901,
        -1.2694,  0.6703,  1.6357,  0.6951,  0.7787], device='cuda:0')
out
tensor([-1.72

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 2.67 GiB already allocated; 0 bytes free; 2.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [97]:
mad

3144.950979232788

In [98]:
fad

5733.886047363281